In [ ]:
# Part of this code is adapted from the the following sources:
# - Kaggle's Time Series Analysis course: https://www.kaggle.com/learn/time-series
# - Kaggle user FilterJoe's Time Series Bonus Lesson: https://www.kaggle.com/code/filterjoe/time-series-bonus-lesson-unofficial/notebook

# Setup notebook
from warnings import simplefilter
from pathlib import Path
import datetime

import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # plotting

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess # for defining time features

from xgboost import XGBRegressor # for hybrid modeling

import pickle # for saving models

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# define training, validation, and test windows
full_train_start_day = datetime.datetime(2017, 1, 1)
full_train_end_day = datetime.datetime(2017, 8, 15)

train_start_day = full_train_start_day
train_end_day = datetime.datetime(2017, 7, 30)

val_start_day = datetime.datetime(2017, 7, 31)
val_end_day = datetime.datetime(2017, 8, 15)

test_start_day = datetime.datetime(2017, 8, 16)
test_end_day = datetime.datetime(2017, 8, 31)

In [ ]:
# define models to be used
mod_1 = LinearRegression() 
mod_2 = XGBRegressor()

#hybrid_forecasting_type = "direct" # hybrid version of train/validate but need to comment out lagged features
hybrid_forecasting_type = "day_by_day_refit_all_days" # forecasts one day at a time and fixes results, can use lagged features

In [ ]:
%%time

# load training set
comp_dir = Path('../input/store-sales-time-series-forecasting')
store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols = ['date', 'store_nbr', 'family', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32'},
    parse_dates=['date'],
    infer_datetime_format=True,
    )

store_sales['date'] = store_sales.date.dt.to_period('D') # convert dates to pandas period dtype

# fill in missing Christmas days
m_index = pd.MultiIndex.from_product([store_sales["store_nbr"].unique(),
                                    store_sales["family"].unique(),
                                    pd.date_range(start="2013-1-1", end="2017-8-15", freq="D").to_period('D')] # to get missing Christmas Days
                                    ,names=["store_nbr","family", "date"])
store_sales = store_sales.set_index(["store_nbr","family", "date"]).reindex(m_index, fill_value=0).sort_index()


store_sales = store_sales.unstack(['store_nbr', 'family']).fillna(0) 
store_sales = store_sales.stack(['store_nbr', 'family'])
store_sales = store_sales[['sales','onpromotion']] # reorder columns to be in the expected order

In [ ]:
# defining a new feature called old_stores that states whether stores were open at the start of data collection or not
df_sales_train = pd.read_csv(comp_dir / 'train.csv', parse_dates=['date'])
df_trans = pd.read_csv(comp_dir / 'transactions.csv', parse_dates=['date'])

total_daily_sales = (
    df_sales_train.drop(columns = 'onpromotion')
    .groupby(['date', 'store_nbr'])
    .sum()
    .reset_index())
    
stores_temp = pd.merge(df_trans, total_daily_sales, on=['date', 'store_nbr']).drop(columns="id")

    
old_stores = stores_temp[stores_temp['date'] == '2013-01-02'].store_nbr.values

In [ ]:
# load in holiday data
comp_dir = Path('../input/store-sales-time-series-forecasting')

holidays_events = pd.read_csv(
    comp_dir / "holidays_events.csv",
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',},
    parse_dates=['date'],
    infer_datetime_format=True,)

holidays_events['date'] = holidays_events['date'].replace({'2013-04-29':pd.to_datetime('2013-03-29')}) # 'Good Friday' mistake correction
holidays_events = holidays_events.set_index('date').to_period('D').sort_index() # note the sort after Good Friday correction

In [ ]:
# create workday calendar, incorporating holidays
# credit to KDJ2020: https://www.kaggle.com/dkomyagin/simple-ts-ridge-rf

calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31')).to_period('D')
calendar['dofw'] = calendar.index.dayofweek

df_hev = holidays_events[holidays_events.locale == 'National'] # National level only for simplicity
df_hev = df_hev.groupby(df_hev.index).first() # Keep one event only

calendar['wd'] = True
calendar.loc[calendar.dofw > 4, 'wd'] = False
calendar = calendar.merge(df_hev, how='left', left_index=True, right_index=True)

calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True

# Transferred column True: holiday officially falls on that calendar day, but was moved to another date by the government.
# type Transfer: day transfer (True) holiday is actually celebrated (i.e. 8/10/17 --> 8/11/17 Primer Grito de Independencia)

# type Bridge: extra days that are added to a holiday (e.g., to extend the break across a long weekend).
# These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.

# type Additional:  days added to a regular calendar holiday such as happens around Christmas

In [ ]:
# define BoostedHybrid class to train and predict using both models

class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None
        self.stack_cols = None
        self.y_resid = None

    def fit1(self, X_1, y, stack_cols=None):
        self.model_1.fit(X_1, y) # train model 1
        y_fit = pd.DataFrame(
            self.model_1.predict(X_1), # predict from model 1
            index=X_1.index,
            columns=y.columns,
        )
        self.y_resid = y - y_fit # residuals from model 1, which X2 may want to access to create lag (or other) features
        self.y_resid = self.y_resid.stack(stack_cols).squeeze()  # wide to long
        
    def fit2(self, X_2, first_n_rows_to_ignore, stack_cols=None):
        self.model_2.fit(X_2.iloc[first_n_rows_to_ignore*1782: , :], self.y_resid.iloc[first_n_rows_to_ignore*1782:]) # Train model_2
        self.y_columns = y.columns # Save for predict method
        self.stack_cols = stack_cols # Save for predict method

    def predict(self, X_1, X_2, first_n_rows_to_ignore):
        y_pred = pd.DataFrame(
            self.model_1.predict(X_1.iloc[first_n_rows_to_ignore: , :]),
            index=X_1.iloc[first_n_rows_to_ignore: , :].index,
            columns=self.y_columns,
        )
        y_pred = y_pred.stack(self.stack_cols).squeeze()  # wide to long
#         display(X_2.iloc[first_n_rows_to_ignore*1782: , :]) # uncomment when debugging
        y_pred += self.model_2.predict(X_2.iloc[first_n_rows_to_ignore*1782: , :]) # Add model_2 predictions to model_1 predictions
        return y_pred.unstack(self.stack_cols)

In [ ]:
# function to make time features for hybrid model 1 using DeterministicProcess
def make_dp_features(df):
    y = df.loc[:, 'sales']
    #fourier_a = CalendarFourier(freq='A', order=4)
    fourier_m = CalendarFourier(freq='M', order=4)
    dp = DeterministicProcess(
        index=y.index,
        constant=True, 
        order=1, # look for a linear trend
        seasonal=True, 
        additional_terms=[fourier_m], # add Fourier pairs for modeling seasonality
        drop=True, # drop linearly dependent terms
    )
    return y, dp

In [ ]:
# create features for hybrid model 1

def make_X1_features(df, start_date, end_date, is_test_set=False):
    if is_test_set:
        X1 = df.rename_axis('date')
    else:
        y, dp = make_dp_features(df)
        X1 = dp.in_sample() # seasonal (weekly) and Fourier (longer time frame) features are generated using DeterministicProcess
    
    
    # other features:
    
    #X1['wage_day'] = (X1.index.day == X1.index.daysinmonth) | (X1.index.day == 15) # wage day features seem better for XGBoost than linear regression
    #X1['wage_day_lag_1'] = (X1.index.day == 1) | (X1.index.day == 16)
    X1['NewYear'] = (X1.index.dayofyear == 1)
    X1['Christmas'] = (X1.index=='2016-12-25') | (X1.index=='2015-12-25') | (X1.index=='2014-12-25') | (X1.index=='2013-12-25')
    
    if is_test_set:
        return X1
    else:
        return X1, y, dp

In [ ]:
# create feature set X2 for hybrid model 2, including helper functions

max_lag = 7 # max_lag is needed so BoostedHybrid class knows how many rows to drop that have NaN or incorrect data after creating lag features
            # should match the number of days for the rolling mean used

def encode_categoricals(df, columns):
    le = LabelEncoder()  # from sklearn.preprocessing
    for col in columns:
        df[col] = le.fit_transform(df[col])
    return df

def make_X2_lags(ts, lags, lead_time=1, name='y', stack_cols=None):
    ts = ts.unstack(stack_cols)
    df = pd.concat(
        {
            f'{name}_lag_{i}': ts.shift(i, freq="D") # freq adds i extra day(s) to end: only one extra day is needed so rest will be dropped
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)
    df = df.stack(stack_cols).reset_index()
    df = encode_categoricals(df, stack_cols)
    df = df.set_index('date').sort_values(by=stack_cols) # return sorted so can correctly compute rolling means (if desired)
    return df

def make_X2_features(df, y_resid):
    stack_columns = ['store_nbr', 'family']
    
    # promo_lag features to try later: 
    shifted_promo_df = make_X2_lags(df.squeeze(), lags=4, name='promo', stack_cols=['store_nbr', 'family'])
    shifted_promo_df['promo_mean_rolling_7'] = shifted_promo_df['promo_lag_1'].rolling(window=7, center=False).mean()
    #shifted_promo_df['promo_median_rolling_91'] = shifted_promo_df['promo_lag_1'].rolling(window=91, center=False).median().fillna(method='bfill')
    #shifted_promo_df['promo_median_rolling_162'] = shifted_promo_df['promo_lag_1'].rolling(window=162, center=False).median().fillna(method='bfill')
    # for rolling window medians, backfilling seems reasonable as medians shouldn't change too much. Trying min_periods produced wacky (buggy?) results
    
    # y_lag features, some features to try later: 
    shifted_y_df = make_X2_lags(y_resid, lags=2, name='y_res', stack_cols=stack_columns)
    shifted_y_df['y_mean_rolling_7'] = shifted_y_df['y_res_lag_1'].rolling(window=7, center=False).mean()
    #shifted_y_df['y_mean_rolling_14'] = shifted_y_df['y_res_lag_1'].rolling(window=14, center=False).mean()
    #shifted_y_df['y_mean_rolling_28'] = shifted_y_df['y_res_lag_1'].rolling(window=28, center=False).mean()
    #shifted_y_df['y_median_rolling_91'] = shifted_y_df['y_res_lag_1'].rolling(window=91, center=False).median().fillna(method='bfill')
    #shifted_y_df['y_median_rolling_162'] = shifted_y_df['y_res_lag_1'].rolling(window=162, center=False).median().fillna(method='bfill')
    
    # other features
    df = df.reset_index(stack_columns)
    X2 = encode_categoricals(df, stack_columns)
    
#     X2["day_of_m"] = X2.index.day  # day of month (label encloded) feature for learning seasonality
#     X2 = encode_categoricals(df, ['day_of_m']) # encoding as categorical has tiny impact with XGBoost
    X2["day_of_w"] = X2.index.dayofweek # does absolutely nothing alone
    X2 = encode_categoricals(df, ['day_of_w'])
    old_stores_strings = list(map(str, old_stores))
    X2['old'] = X2['store_nbr'].isin(old_stores_strings) # True if store had existing sales prior to training time period   
    #X2['wage_day'] = (X2.index.day == X2.index.daysinmonth) | (X2.index.day == 15) # is it bad to have this in both X1 AND X2?
    #X2['wage_day_lag_1'] = (X2.index.day == 1) | (X2.index.day == 16)
    #X2['promo_mean'] = X2.groupby(['store_nbr', 'family'])['onpromotion'].transform("mean") + 0.000001
    #X2['promo_ratio'] = X2.onpromotion / (X2.groupby(['store_nbr', 'family'])['onpromotion'].transform("mean") + 0.000001)

    # Can experiment with dropping basic feature but keeping something computed from it (i.e. drop y_res_lag_1 feature, keeping y_mean_rolling_7
#     shifted_y_df.drop('y_res_lag_1', axis=1, inplace=True)
#     shifted_promo_df.drop('promo_lag_1', axis=1, inplace=True)
#     X2.drop('onpromotion', axis=1, inplace=True)

    # if removing all lag features, then comment out the following two lines
    X2 = X2.merge(shifted_y_df, on=['date', 'store_nbr', 'family'], how='left')
    X2 = X2.merge(shifted_promo_df, on=['date', 'store_nbr', 'family'], how='left') # merges work if they are last line before return
    return X2

In [ ]:
%time

# quick test for quickly re-training model

# unstack pivots MultiIndex to 54 x 33 = 1782 y columns
store_sales_in_date_range = store_sales.unstack(['store_nbr', 'family']).loc[full_train_start_day:full_train_end_day]

model = BoostedHybrid(model_1=mod_1, model_2=mod_2) # Boosted Hybrid

X_1, y, dp = make_X1_features(store_sales_in_date_range, full_train_start_day, full_train_end_day) # preparing X1 for hybrid model 1
model.fit1(X_1, y, stack_cols=['store_nbr', 'family']) # fit1 before make_X2_features, since X2 may want to create lag features from model.y_resid
X_2 = make_X2_features(store_sales_in_date_range # preparing X2 for hybrid model 2
                       .drop('sales', axis=1)
                       .stack(['store_nbr', 'family']),
                       model.y_resid)
model.fit2(X_2, max_lag, stack_cols=['store_nbr', 'family'])

y_pred = model.predict(X_1, X_2, max_lag).clip(0.0)

In [ ]:
# train/val split prep that is the same for any of the hybrid forecasting methods

training_days = (train_end_day - train_start_day).days + 1
validation_days = (val_end_day - val_start_day).days + 1
print("Training data set (excluding validation days) has", training_days, "days")
print("Validation data set has", validation_days, "days\n")

store_sales_in_date_range = store_sales.unstack(['store_nbr', 'family']).loc[train_start_day:train_end_day]
store_data_in_val_range = store_sales.unstack(['store_nbr', 'family']).loc[val_start_day:val_end_day]
y_val = y[val_start_day:val_end_day] # use y to evaluate validation set, though we will treat y as unknown when training

model_for_val = BoostedHybrid(model_1=mod_1, model_2=mod_2)

In [ ]:
def truncateFloat(data):
    return tuple( ["{0:.2f}".format(x) if isinstance(x,float) else (x if not isinstance(x,tuple) else truncateFloat(x)) for x in data])

temp = X_2[(X_2.store_nbr == 1) & (X_2.family == 3)]
temp.iloc[max_lag: , :].apply(lambda s: truncateFloat(s)) # comment out next line if don't want to see nan rows

temp.apply(lambda s: truncateFloat(s)).head(10) # note that the fit method of BoostedHybrid class skips over nan rows

In [ ]:
%%time

# DIRECT hybrid version of train/validate (can't use with lagged/rolling features on y, promo, etc.)
if hybrid_forecasting_type == "direct":
    X_1_train, y_train, dp_val = make_X1_features(store_sales_in_date_range, train_start_day, train_end_day) # preparing X1 for hybrid part 1: LinearRegression
    model_for_val.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family']) # fit1 before make_X2_features, since X2 may want to create lag features from model.y_resid
    X_2_train = make_X2_features(store_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_val.y_resid) # preparing X2 for hybrid part 2: XGBoost
    model_for_val.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])

    X_1_val = make_X1_features(dp_val.out_of_sample(steps=validation_days), val_start_day, val_end_day, is_test_set=True)
    X_2_val = make_X2_features(store_data_in_val_range
                                .drop('sales', axis=1)
                                .stack(['store_nbr', 'family']),
                                model_for_val.y_resid) # preparing X2 for hybrid part 2: XGBoost
    y_fit = model_for_val.predict(X_1_train, X_2_train, max_lag).clip(0.0)
    y_pred = model_for_val.predict(X_1_val, X_2_val, 0).clip(0.0) # set max_lag to 0 because need entire time span for validation data set
    
    if type(model_for_val.model_2) == XGBRegressor:
        pickle.dump(model_for_val.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
%%time

# forecast 1 day at a time which allows use of lag features. Slow.
# Each new day y is fixed after it's initial prediction - no y row is ever predicted more than once
if hybrid_forecasting_type == "day_by_day_fixed_past":
    #initial fit on train portion of train/val split
    X_1_train, y_train, dp_val = make_X1_features(store_sales_in_date_range, train_start_day, train_end_day) # preparing X1 for hybrid part 1: LinearRegression
    model_for_val.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family']) # fit1 before make_X2_features, since X2 may want to create lag features from model.y_resid
    X_2_train = make_X2_features(store_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_val.y_resid) # preparing X2 for hybrid part 2: XGBoost
    model_for_val.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])
    y_fit = model_for_val.predict(X_1_train, X_2_train, max_lag).clip(0.0)

    y_pred_combined = y_fit.copy() # initialize y_pred_combined

    # loop through forecast, one day ("step") at a time
    dp_for_full_X1_val_date_range = dp_val.out_of_sample(steps=validation_days)
    for step in range(validation_days):
        dp_steps_so_far = dp_for_full_X1_val_date_range.loc[val_start_day:val_start_day+pd.Timedelta(days=step),:]

        X_1_combined_dp_data = pd.concat([dp_val.in_sample(), dp_steps_so_far])
        X_2_combined_data = pd.concat([store_sales_in_date_range,
                                       store_data_in_val_range.loc[val_start_day:val_start_day+pd.Timedelta(days=step), :]])
        X_1_val = make_X1_features(X_1_combined_dp_data, train_start_day, val_start_day+pd.Timedelta(days=step), is_test_set=True)
        X_2_val = make_X2_features(X_2_combined_data
                                    .drop('sales', axis=1)
                                    .stack(['store_nbr', 'family']),
                                    model_for_val.y_resid) # preparing X2 for hybrid part 2: XGBoost

    #     print("last 3 rows of X_1_val: ")
    #     display(X_1_val.tail(3))
    #     temp_val2 = X_2_val[(X_2_val.store_nbr == 1) & (X_2_val.family == 3)]
    #     print("last 3 rows of X_2_val: ")
    #     display(temp_val2.tail(3).apply(lambda s: truncateFloat(s)))

        y_pred_combined = pd.concat([y_pred_combined,
                                     model_for_val.predict(X_1_val, X_2_val, max_lag).clip(0.0).iloc[-1:]
                                    ])
    #     print("last 3 rows of y_combined: ")
    #     display(y_pred_combined.tail(3).apply(lambda s: truncateFloat(s)))
        y_plus_y_val = pd.concat([y_train, y_pred_combined.iloc[-(step+1):]]) # add newly predicted rows of y_pred_combined
        model_for_val.fit1(X_1_val, y_plus_y_val, stack_cols=['store_nbr', 'family']) # fit on new combined X, y - note that fit prior to val date range will change slightly
        model_for_val.fit2(X_2_val, max_lag, stack_cols=['store_nbr', 'family'])

        rmsle_valid = mean_squared_log_error(y_val.iloc[step:step+1], y_pred_combined.iloc[-1:]) ** 0.5
        print(f'Validation RMSLE: {rmsle_valid:.5f}', "for", val_start_day+pd.Timedelta(days=step))

    y_pred = y_pred_combined[val_start_day:val_end_day]
    print("\ny_pred: ")
    display(y_pred.apply(lambda s: truncateFloat(s)))
    
    if type(model_for_val.model_2) == XGBRegressor:
        pickle.dump(model_for_val.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
%%time
# forecast 1 day at a time which allows use of lag features. Slow.
# each new forecast causes y for all days in training and test (or validation set) to be reforecast.
if hybrid_forecasting_type == "day_by_day_refit_all_days":
    #initial fit on train portion of train/val split
    X_1_train, y_train, dp_val = make_X1_features(store_sales_in_date_range, train_start_day, train_end_day) # preparing X1 for hybrid part 1: LinearRegression
    model_for_val.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family']) # fit1 before make_X2_features, since X2 may want to create lag features from model.y_resid
    X_2_train = make_X2_features(store_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_val.y_resid) # preparing X2 for hybrid part 2: XGBoost
    model_for_val.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])
    y_fit = model_for_val.predict(X_1_train, X_2_train, max_lag).clip(0.0)

    # loop through forecast, one day ("step") at a time
    dp_for_full_X1_val_date_range = dp_val.out_of_sample(steps=validation_days)
    for step in range(validation_days):
        dp_steps_so_far = dp_for_full_X1_val_date_range.loc[val_start_day:val_start_day+pd.Timedelta(days=step),:]

        X_1_combined_dp_data = pd.concat([dp_val.in_sample(), dp_steps_so_far])
        X_2_combined_data = pd.concat([store_sales_in_date_range,
                                       store_data_in_val_range.loc[val_start_day:val_start_day+pd.Timedelta(days=step), :]])
        X_1_val = make_X1_features(X_1_combined_dp_data, train_start_day, val_start_day+pd.Timedelta(days=step), is_test_set=True)
        X_2_val = make_X2_features(X_2_combined_data
                                    .drop('sales', axis=1)
                                    .stack(['store_nbr', 'family']),
                                    model_for_val.y_resid) # preparing X2 for hybrid part 2: XGBoost

    #     print("last 3 rows of X_1_val: ")
    #     display(X_1_val.tail(3))
    #     temp_val2 = X_2_val[(X_2_val.store_nbr == 1) & (X_2_val.family == 3)]
    #     print("last 3 rows of X_2_val: ")
    #     display(temp_val2.tail(3).apply(lambda s: truncateFloat(s)))

        y_pred_combined = model_for_val.predict(X_1_val, X_2_val, max_lag).clip(0.0) # generate y with 
    #     print("last 3 rows of y_combined: ")
    #     display(y_pred_combined.tail(3).apply(lambda s: truncateFloat(s)))
        y_plus_y_val = pd.concat([y_train, y_pred_combined.iloc[-(step+1):]]) # add newly predicted rows of y_pred_combined
        model_for_val.fit1(X_1_val, y_plus_y_val, stack_cols=['store_nbr', 'family']) # fit on new combined X, y - note that fit prior to val date range will change slightly
        model_for_val.fit2(X_2_val, max_lag, stack_cols=['store_nbr', 'family'])

        rmsle_valid = mean_squared_log_error(y_val.iloc[step:step+1], y_pred_combined.iloc[-1:]) ** 0.5
        print(f'Validation RMSLE: {rmsle_valid:.5f}', "for", val_start_day+pd.Timedelta(days=step))
    #     print("end of round ", step)

    y_pred = y_pred_combined[val_start_day:val_end_day]
    print("\ny_pred: ")
    display(y_pred.apply(lambda s: truncateFloat(s)))
    
    if type(model_for_val.model_2) == XGBRegressor:
        pickle.dump(model_for_val.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
rmsle_train = mean_squared_log_error(y_train.iloc[max_lag: , :].clip(0.0), y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_val.clip(0.0), y_pred) ** 0.5
print()
print(f'Training RMSLE: {rmsle_train:.5f}')
print(f'Validation RMSLE: {rmsle_valid:.5f}')
    
y_predict = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y_val.stack(['store_nbr', 'family']).reset_index().copy()
y_target.rename(columns={y_target.columns[3]:'sales'}, inplace=True)
y_target['sales_pred'] = y_predict[0].clip(0.0) # Sales should be >= 0
y_target['store_nbr'] = y_target['store_nbr'].astype(int)

print('\nValidation RMSLE by family')
display(y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])))

print('\nValidation RMSLE by store')
display(y_target.sort_values(by="store_nbr").groupby('store_nbr').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])))

In [ ]:
# load in the test set
df_test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
df_test

In [ ]:
# train/test split prep that is the same for any of the hybrid forecasting methods
train_days = (full_train_end_day - full_train_start_day).days + 1
test_days = (test_end_day - test_start_day).days + 1

print("data trained over", train_days, "days")
print("test forecasting period is", test_days, "days through", test_end_day, "\n")
store_sales_in_date_range = store_sales.unstack(['store_nbr', 'family']).loc[full_train_start_day:full_train_end_day]
store_data_in_test_range = df_test.unstack(['store_nbr', 'family']).drop('id', axis=1)

# previously prepared data and fit "model" from data ranging from full_train_start_day to full_train_end_day. Can be used by when fitting test.
model_for_test = BoostedHybrid(model_1=mod_1, model_2=mod_2)

In [ ]:
%%time
# DIRECT hybrid version of train/validate (can't use with lagged/rolling features on y, promo, etc.)
# model.fit (1 and 2) already happened previously (on training set with a time range ending on most recent date)

if hybrid_forecasting_type == "direct":
    X_1_test = make_X1_features(dp.out_of_sample(steps=int(test_days)), test_start_day, test_end_day, is_test_set=True)
    X_2_test = make_X2_features(store_data_in_test_range.loc[test_start_day:test_end_day]
                                .stack(['store_nbr', 'family']),
                                model.y_resid) # preparing X2 for hybrid part 2: XGBoost
    y_forecast = pd.DataFrame(model.predict(X_1_test, X_2_test, 0).clip(0.0), index=X_1_test.index, columns=y.columns) # set max_lag to 0 because need entire time span for test data set
    
    if type(model.model_2) == XGBRegressor:
        pickle.dump(model.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
%%time
# forecast 1 day at a time which allows use of lag features. Slow.
# Each new day y is fixed after it's initial prediction - no y row is ever predicted more than once
if hybrid_forecasting_type == "day_by_day_fixed_past":
    #initial fit on train portion of train/test split
    X_1_train, y_train, dp_test = make_X1_features(store_sales_in_date_range, full_train_start_day, full_train_end_day) # preparing X1 for hybrid part 1: LinearRegression
    model_for_test.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family']) # fit1 before make_X2_features, since X2 may want to create lag features from model.y_resid
    X_2_train = make_X2_features(store_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_test.y_resid) # preparing X2 for hybrid part 2: XGBoost
    model_for_test.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])

    y_forecast_combined = model_for_test.predict(X_1_train, X_2_train, max_lag).clip(0.0) # initializing with training set fit

    dp_for_full_X1_test_date_range = dp_test.out_of_sample(steps=test_days)
    for step in range(test_days):
        dp_steps_so_far = dp_for_full_X1_test_date_range.loc[test_start_day:test_start_day+pd.Timedelta(days=step),:]

        X_1_combined_dp_data = pd.concat([dp_test.in_sample(), dp_steps_so_far])
        X_2_combined_data = pd.concat([store_sales_in_date_range,
                                       store_data_in_test_range.loc[test_start_day:test_start_day+pd.Timedelta(days=step), :]])
        X_1_test = make_X1_features(X_1_combined_dp_data, train_start_day, test_start_day+pd.Timedelta(days=step), is_test_set=True)
        X_2_test = make_X2_features(X_2_combined_data
                                    .drop('sales', axis=1)
                                    .stack(['store_nbr', 'family']),
                                    model_for_test.y_resid) # preparing X2 for hybrid part 2: XGBoost
    #     print("last 3 rows of X_1_test: ")
    #     display(X_1_test.tail(3))
    #     temp_test2 = X_2_test[(X_2_test.store_nbr == 1) & (X_2_test.family == 3)]
    #     print("last 3 rows of X_2_test: ")
    #     display(temp_test2.tail(3).apply(lambda s: truncateFloat(s)))

        y_forecast_combined = pd.concat([y_forecast_combined,
                                     model_for_test.predict(X_1_test, X_2_test, max_lag).clip(0.0).iloc[-1:]
                                    ])   
    #     print("last 3 rows of y_forecast_combined: ")
    #     display(y_forecast_combined.tail(3).apply(lambda s: truncateFloat(s)))

        y_plus_y_test = pd.concat([y_train, y_forecast_combined.iloc[-(step+1):]]) # add newly predicted (last step+1) rows of y_test_combined
        model_for_test.fit1(X_1_test, y_plus_y_test, stack_cols=['store_nbr', 'family']) # fit on new combined X, y - note that fit prior to test date range will change slightly
        model_for_test.fit2(X_2_test, max_lag, stack_cols=['store_nbr', 'family'])
        print("finished forecast for", test_start_day+pd.Timedelta(days=step))

    display(y_forecast_combined[test_start_day:test_end_day])
    y_forecast = pd.DataFrame(y_forecast_combined[test_start_day:test_end_day].clip(0.0), index=X_1_test.index, columns=y.columns)
    print('\nFinished creating test set forecast\n')
    
    if type(model_for_test.model_2) == XGBRegressor:
        pickle.dump(model_for_test.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
%%time
# forecast 1 day at a time which allows use of lag features. Slow.
# each new forecast causes y for all days in training and test (or validation set) to be reforecast.
if hybrid_forecasting_type == "day_by_day_refit_all_days":
    #initial fit on train portion of train/test split
    X_1_train, y_train, dp_test = make_X1_features(store_sales_in_date_range, full_train_start_day, full_train_end_day) # preparing X1 for hybrid part 1: LinearRegression
    model_for_test.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family']) # fit1 before make_X2_features, since X2 may want to create lag features from model.y_resid
    X_2_train = make_X2_features(store_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_test.y_resid) # preparing X2 for hybrid part 2: XGBoost
    model_for_test.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])
    # y_full_train = model_for_test.predict(X_1_train, X_2_train, max_lag).clip(0.0) # do I need this line here?


    dp_for_full_X1_test_date_range = dp_test.out_of_sample(steps=test_days)
    for step in range(test_days):
        dp_steps_so_far = dp_for_full_X1_test_date_range.loc[test_start_day:test_start_day+pd.Timedelta(days=step),:]

        X_1_combined_dp_data = pd.concat([dp_test.in_sample(), dp_steps_so_far])
        X_2_combined_data = pd.concat([store_sales_in_date_range,
                                       store_data_in_test_range.loc[test_start_day:test_start_day+pd.Timedelta(days=step), :]])
        X_1_test = make_X1_features(X_1_combined_dp_data, train_start_day, test_start_day+pd.Timedelta(days=step), is_test_set=True)
        X_2_test = make_X2_features(X_2_combined_data
                                    .drop('sales', axis=1)
                                    .stack(['store_nbr', 'family']),
                                    model_for_test.y_resid) # preparing X2 for hybrid part 2: XGBoost
    #     print("last 3 rows of X_1_test: ")
    #     display(X_1_test.tail(3))
    #     temp_test2 = X_2_test[(X_2_test.store_nbr == 1) & (X_2_test.family == 3)]
    #     print("last 3 rows of X_2_test: ")
    #     display(temp_test2.tail(3).apply(lambda s: truncateFloat(s)))

        y_forecast_combined = model_for_test.predict(X_1_test, X_2_test, max_lag).clip(0.0) # generate y with 

    #     print("last 3 rows of y_forecast_combined: ")
    #     display(y_forecast_combined.tail(3).apply(lambda s: truncateFloat(s)))

        y_plus_y_test = pd.concat([y_train, y_forecast_combined.iloc[-(step+1):]]) # add newly predicted (last step+1) rows of y_test_combined
        model_for_test.fit1(X_1_test, y_plus_y_test, stack_cols=['store_nbr', 'family']) # fit on new combined X, y - note that fit prior to test date range will change slightly
        model_for_test.fit2(X_2_test, max_lag, stack_cols=['store_nbr', 'family'])
        print("finished forecast for", test_start_day+pd.Timedelta(days=step))

    display(y_forecast_combined[test_start_day:test_end_day])

    y_forecast = pd.DataFrame(y_forecast_combined[test_start_day:test_end_day].clip(0.0), index=X_1_test.index, columns=y.columns)
    print('\nFinished creating test set forecast\n')
    
    if type(model_for_test.model_2) == XGBRegressor:
        pickle.dump(model_for_test.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
#X_test = dp.out_of_sample(steps = 16)
#X_test['NewYearsDay'] = (X_test.index.dayofyear == 1)

#X_test

In [ ]:
# make predictions for test set
# will use a dictionary to store id:sales for every test value
#test_dict = {}
       
# generate features for 16-day forecast window
#X_test = dp.out_of_sample(steps = 16)
#X_test['NewYearsDay'] = (X_test.index.dayofyear == 1)

# loop over all stores
#for store_nbr, test_0 in test.groupby(level = 0):
    
    # loop over all sales categories
#    for family, test_1 in test_0.groupby(level = 1):
        
        # pick current category
#        test_family = test_1.index.get_level_values(1)[0]
    
        # fetch linear model for detrend & deseason
#        model_1 = LinearRegression()
#        model_1.coef_ = model_params[test_family]
#        model_1.intercept_ = intercepts[test_family]
#        model_1.feature_names_in_ = X_test.columns
        
        # predict sales for forecast window, output is a list
#        y_test1 = model_1.predict(X_test)
        
        # set up features for XGBoost
        #promo = test_1.onpromotion.reset_index(drop = True) # grab onpromotion data
        
#        day = pd.Series(X_test.index.day) # grab 'day' values
#        X_test_xgb = pd.concat([pd.Series(test_1.index.get_level_values(1)), promo, day], axis = 1,  ignore_index = True) # concatenate the above with current family 
#        X_test_xgb = X_test_xgb.rename(columns = {0:'family', 1:'onpromotion', 2:'day'}) # rename columns
#        X_test_xgb['family'] = le.transform(X_test_xgb['family']) # encode 'family'
#        X_test_xgb.index = X_test.index # set index

        # predict residual seasonality and other effects with XGBoost
#        y_test2 = model_xgb.predict(X_test_xgb)
        
        # combine models
#        y_test = y_test1 + y_test2
#        y_test = np.where(y_test<0, 0, y_test) # metric is RMSLE, replace negative values to avoid log errors
        
        # grab test ids, output is a list
#        keys = test_1.id.to_string(header = False, index = False).split()
        
        # store ids and inferences in a dictionary
#        for i, key in enumerate(keys):
#            value = y_test[i]
#            test_dict[key] = value
            
# convert dictionary of inferences to DataFrame to write to .csv
#test_sub = pd.DataFrame.from_dict(test_dict, orient = 'index', columns = ['sales'])
#test_sub.index.name = 'id' 
#test_sub = test_sub.sort_index(ascending = True)

#test_sub.to_csv('submission.csv')

In [ ]:
# plot style settings from learntools.time_series.style

plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

In [ ]:
# see example predictions (both validation and test data sets) for a specific store/family:

STORE_NBR = '1'  # 1 - 54
FAMILY = 'BEVERAGES' # display(store_sales.index.get_level_values('family').unique())

ax = y.loc(axis=1)[STORE_NBR, FAMILY].plot(**plot_params, figsize=(16, 4))
ax = y_pred.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax, marker='.', color='red', markersize=12) # markers: big size for tiny validation sets (1-2 days)
ax = y_forecast.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax, marker='.', color='orange', markersize=12)
ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

In [ ]:
# creates submission file submission.csv

y_submit = y_forecast.stack(['store_nbr', 'family'])
y_submit = pd.DataFrame(y_submit, columns=['sales'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)

In [ ]:
y_submit